Image Preprocessing without background removal

In [ ]:
import cv2
import numpy as np
import os
import random

# Define input and output directory paths
input_base_folder = "../skinType"  # Parent folder containing 'oily', 'dry', 'normal'
output_base_folder = "./Processed_Images/"

# Fixed patch size
PATCH_SIZE = 256
MIN_SKIN_AREA_RATIO = 0.9  # Minimum 20% of the image should be skin

def detect_skin(image):
    """Detect skin using HSV and YCrCb color spaces."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Define skin color range in HSV
    lower_hsv = np.array([0, 20, 70], dtype=np.uint8)
    upper_hsv = np.array([20, 255, 255], dtype=np.uint8)
    
    # Define skin color range in YCrCb
    lower_ycrcb = np.array([0, 133, 77], dtype=np.uint8)
    upper_ycrcb = np.array([255, 173, 127], dtype=np.uint8)
    
    # Apply masks
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)
    
    # Combine masks
    combined_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)
    
    # Apply Otsu thresholding
    _, otsu_mask = cv2.threshold(combined_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    clean_mask = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    
    return clean_mask

def extract_skin(image, mask):
    """Find the largest skin region, ensure it meets the minimum skin area threshold."""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None  # No skin detected
    
    # Find the largest skin region
    largest_contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest_contour)
    
    # Calculate image size
    img_height, img_width = image.shape[:2]
    total_pixels = img_height * img_width
    
    # Ignore images where skin area is less than 20% of total pixels
    if area / total_pixels < MIN_SKIN_AREA_RATIO:
        return None
    
    # Get bounding box of skin region
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Crop skin region
    skin_crop = image[y:y+h, x:x+w]
    
    # Ensure minimum size for cropping
    if w < PATCH_SIZE or h < PATCH_SIZE:
        return None  # Skip small regions
    
    return skin_crop

def extract_patches(skin_region):
    """Generate a random number of patches from the skin region."""
    patches = []
    h, w = skin_region.shape[:2]
    
    # Randomly choose the number of patches (between 2 to 6)
    num_patches = random.randint(2, 6)
    
    for _ in range(num_patches):
        # Select random starting points for patches
        x_start = random.randint(0, max(1, w - PATCH_SIZE))
        y_start = random.randint(0, max(1, h - PATCH_SIZE))
        
        patch = skin_region[y_start:y_start + PATCH_SIZE, x_start:x_start + PATCH_SIZE]
        
        # Ensure the patch is of correct size before adding
        if patch.shape[0] == PATCH_SIZE and patch.shape[1] == PATCH_SIZE:
            patches.append(patch)
    
    return patches

# Process each category folder (oily, dry, normal)
for category in ["oily", "dry", "normal"]:
    input_folder = os.path.join(input_base_folder, category)
    output_folder = os.path.join(output_base_folder, category)
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)
            
            if image is None:
                continue  # Skip unreadable images
            
            skin_mask = detect_skin(image)
            skin_region = extract_skin(image, skin_mask)
            
            if skin_region is not None:  # Save only if enough skin is detected
                patches = extract_patches(skin_region)
                for i, patch in enumerate(patches):
                    output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_patch{i}.jpg")
                    cv2.imwrite(output_path, patch)

print("Processing complete. Only valid skin images saved.")


Image preprocessing with background removal

In [ ]:
import cv2
import numpy as np
import os
import random

# Detect skin using HSV + YCrCb color spaces
def get_skin_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # HSV range
    lower_hsv = np.array([0, 30, 60], dtype=np.uint8)
    upper_hsv = np.array([20, 150, 255], dtype=np.uint8)
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)

    # YCrCb range
    lower_ycrcb = np.array((0, 133, 77), dtype=np.uint8)
    upper_ycrcb = np.array((255, 173, 127), dtype=np.uint8)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)

    # Combine both
    skin_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)

    # Clean mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=1)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

    return skin_mask

# Check if patch has RGB skin-like color range
def is_patch_skin_colored(patch):
    mean_color = cv2.mean(patch)[:3]
    b, g, r = mean_color
    return (
        80 < r < 255 and
        30 < g < 200 and
        20 < b < 170 and
        r > g and r > b and abs(r - g) > 15
    )

# Optional: reject blurry patches
def is_blurry(patch, threshold=100.0):
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold

# Extract skin-only patches from image
def extract_skin_patches(image, mask, patch_size=(100, 100), num_patches=4, min_skin_ratio=0.4):
    coords = cv2.findNonZero(mask)
    if coords is None:
        return []

    h, w = image.shape[:2]
    patches = []
    attempts = 0

    while len(patches) < num_patches and attempts < num_patches * 20:
        x, y = random.choice(coords)[0]
        x = max(0, min(x - patch_size[0] // 2, w - patch_size[0]))
        y = max(0, min(y - patch_size[1] // 2, h - patch_size[1]))

        patch = image[y:y+patch_size[1], x:x+patch_size[0]]
        patch_mask = mask[y:y+patch_size[1], x:x+patch_size[0]]

        if patch.shape[:2] != patch_size:
            attempts += 1
            continue

        skin_ratio = np.sum(patch_mask == 255) / (patch_size[0] * patch_size[1])
        if skin_ratio < min_skin_ratio:
            attempts += 1
            continue

        if not is_patch_skin_colored(patch):
            attempts += 1
            continue

        if is_blurry(patch):
            attempts += 1
            continue

        patches.append(patch)
        attempts += 1

    return patches

# Main processor
def process_skin_classes(input_dir, output_dir):
    classes = ['oily', 'dry', 'normal']

    for cls in classes:
        input_cls_path = os.path.join(input_dir, cls)
        output_cls_path = os.path.join(output_dir, cls)
        os.makedirs(output_cls_path, exist_ok=True)

        for filename in os.listdir(input_cls_path):
            img_path = os.path.join(input_cls_path, filename)
            image = cv2.imread(img_path)

            if image is None:
                print(f"Skipped unreadable file: {img_path}")
                continue

            skin_mask = get_skin_mask(image)
            patches = extract_skin_patches(image, skin_mask)

            if not patches:
                print(f"No good patches from {filename} in class {cls}")
                continue

            for i, patch in enumerate(patches):
                patch_name = f"{os.path.splitext(filename)[0]}_patch{i}.jpg"
                patch_path = os.path.join(output_cls_path, patch_name)
                cv2.imwrite(patch_path, patch)

            print(f"Processed {filename} ({len(patches)} patches) from class {cls}")

# Run it
if __name__ == "__main__":
    input_folder = "../skinType"             # Folder with oily/, dry/, normal/
    output_folder = "./Processed_Images_Clean/"  # Save cleaned patches here
    process_skin_classes(input_folder, output_folder)


CAnny Edge detector

In [ ]:
import cv2
import numpy as np
import os

def get_skin_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    lower_hsv = np.array([0, 30, 60], dtype=np.uint8)
    upper_hsv = np.array([20, 150, 255], dtype=np.uint8)
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)

    lower_ycrcb = np.array((0, 133, 77), dtype=np.uint8)
    upper_ycrcb = np.array((255, 173, 127), dtype=np.uint8)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)

    skin_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=1)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

    return skin_mask

def process_skin_classes(input_dir, output_dir):
    classes = ['oily', 'dry', 'normal']

    for cls in classes:
        input_cls_path = os.path.join(input_dir, cls)
        output_cls_path = os.path.join(output_dir, cls)
        os.makedirs(output_cls_path, exist_ok=True)

        for filename in os.listdir(input_cls_path):
            img_path = os.path.join(input_cls_path, filename)
            image = cv2.imread(img_path)

            if image is None:
                print(f"Skipped unreadable file: {img_path}")
                continue

            skin_mask = get_skin_mask(image)

            # Optional: Combine mask with original image to visualize
            skin_result = cv2.bitwise_and(image, image, mask=skin_mask)

            # Save the result
            result_path = os.path.join(output_cls_path, f"{os.path.splitext(filename)[0]}_skin.jpg")
            cv2.imwrite(result_path, skin_result)

            print(f"Processed {filename} (full image skin detection)")

if __name__ == "__main__":
    input_folder = "../skinType"
    output_folder = "./Processed_Images_SkinOnly/"
    process_skin_classes(input_folder, output_folder)


In [5]:
import cv2
import numpy as np
import os

# Load OpenCV's pre-trained Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_crop_face(image, size=(256, 256)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        return None  # No face found

    # Choose the largest face (in case of multiple)
    x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]

    # Optional: Add some padding
    pad = int(0.2 * max(w, h))
    x = max(x - pad, 0)
    y = max(y - pad, 0)
    w = min(w + 2*pad, image.shape[1] - x)
    h = min(h + 2*pad, image.shape[0] - y)

    face_crop = image[y:y+h, x:x+w]
    resized_face = cv2.resize(face_crop, size)

    return resized_face

def process_skin_classes(input_dir, output_dir):
    classes = ['oily', 'dry', 'normal']

    for cls in classes:
        input_cls_path = os.path.join(input_dir, cls)
        output_cls_path = os.path.join(output_dir, cls)
        os.makedirs(output_cls_path, exist_ok=True)

        for filename in os.listdir(input_cls_path):
            img_path = os.path.join(input_cls_path, filename)
            image = cv2.imread(img_path)

            if image is None:
                print(f"Skipped unreadable file: {img_path}")
                continue

            face_crop = detect_and_crop_face(image)

            if face_crop is None:
                print(f"No face detected in {filename}")
                continue

            output_path = os.path.join(output_cls_path, f"{os.path.splitext(filename)[0]}_face.jpg")
            cv2.imwrite(output_path, face_crop)
            print(f"Processed {filename} -> face crop saved")

if __name__ == "__main__":
    input_folder = "../Preprocessing/Processed_Images_SkinOnly"
    output_folder = "./Processed_Images_FaceCrops_256/"
    process_skin_classes(input_folder, output_folder)


No face detected in oily (1)_skin.jpg
Processed oily (10)_skin.jpg -> face crop saved
Processed oily (100)_skin.jpg -> face crop saved
Processed oily (101)_skin.jpg -> face crop saved
Processed oily (102)_skin.jpg -> face crop saved
Processed oily (103)_skin.jpg -> face crop saved
Processed oily (104)_skin.jpg -> face crop saved
Processed oily (105)_skin.jpg -> face crop saved
Processed oily (106)_skin.jpg -> face crop saved
Processed oily (107)_skin.jpg -> face crop saved
No face detected in oily (108)_skin.jpg
No face detected in oily (109)_skin.jpg
No face detected in oily (11)_skin.jpg
No face detected in oily (110)_skin.jpg
No face detected in oily (111)_skin.jpg
Processed oily (112)_skin.jpg -> face crop saved
No face detected in oily (113)_skin.jpg
Processed oily (114)_skin.jpg -> face crop saved
Processed oily (115)_skin.jpg -> face crop saved
Processed oily (116)_skin.jpg -> face crop saved
Processed oily (117)_skin.jpg -> face crop saved
Processed oily (118)_skin.jpg -> face 